In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
from urllib2 import urlopen # python2
import urllib2
import time

In [2]:
df_sider_indication_raw = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/meddra_all_indications_header.tsv")

In [5]:
df_sider_indication_pt = df_sider_indication_raw[df_sider_indication_raw.MedDRA_concept_type == 'PT']

In [25]:
df_sider_indication = df_sider_indication_pt[['STITCH_compound_id_flat', 'UMLS_concept_id_for_MedDRA','MedDRA_concept_name']].drop_duplicates()

In [28]:
df_stitch = pd.read_table("/DAS_Storage1/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv")

In [57]:
print len(df_sider_indication) # sider의 indication 총 개수
print len(df_sider_indication['STITCH_compound_id_flat'].drop_duplicates()) # indication 정보를 가지고 있는 compound의 개수-->1437개

# stitch_pc id mapping 파일에서 sider에 indication 정보가 존재하는 stitch id를 솎아낸다.
df_query = df_stitch[df_stitch['flat_chemical'].isin(df_sider_indication['STITCH_compound_id_flat'])] 
print len(df_query) # stitch id와 mapping되는 pc의 개수(pc는 중복 없음)
print len(df_query.drop_duplicates('flat_chemical')) #stitch_id의 개수 

15083
1437
45598
1437


In [ ]:
df_query.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_0_query_pubchemID.tsv", sep='\t', index=False)

In [2]:
df_query = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_0_query_pubchemID.tsv")

In [3]:
print len(df_query)
print len(df_query.drop_duplicates('flat_chemical'))
df_query.head(1)

45598
1437


,flat_chemical,stereo_chemical,source_name,source_id
0,CID100002735,CID010000117,PC,10000117


In [10]:
# using UniChem REST API, convert pubchemID to DrugBank ID
results = pd.DataFrame()
  
start = 34200
end = 45598
cnt = start
url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
print'%d : ' % cnt + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in df_query[start:end].iterrows():
    record = row.tolist()
    stitch_id_flat = record[0]
    stitch_id_stereo = record[1]
    pubchem_id = record[3]
    url = url_base + str(pubchem_id) + '/22/2'
        
    MAX_ATTEMPTS = 8
    for attempt in range(MAX_ATTEMPTS):
        try:
            f = urlopen(url)
        except urllib2.HTTPError, e:
            print row
            if e.code == 404:
                cnt = cnt + 1
                if cnt % 1000 == 0 :
                    print '%d : '% cnt + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                break
            else:
                print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                break
                #raise
        except urllib2.URLError, e:
            if e.args[0][0] == 110: # connection time out
                sleep_secs = attempt ** 2
                print e , url
                print '  cnt = %d, error time : ' % (cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print '    Retrying in %d seconds...' % sleep_secs
                time.sleep(sleep_secs)
                continue 
            else:
                print row
                print e
                print '   cnt : %d, error time : ' % (cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                cnt = cnt +1
                break
        else: # error가 일어나지 않을 때          
            all_data = f.readlines()
            if all_data[0] == '[]':
                cnt = cnt + 1
            else:
                db_id=all_data[0] 
                temp_df = pd.DataFrame(data = [[stitch_id_flat, stitch_id_stereo, pubchem_id, db_id]], 
                                       columns = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID'])  
                results = results.append(temp_df)
                cnt = cnt + 1
            if cnt % 1000 == 0 :
                print '%d : '% cnt + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            break
    else:#connection을 8번 시도했는데 안될 경우
        print 'we failed to reconnect 8 times in %s' % url
            
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
results.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/11_1_raw_SIDER_indication_DrugBank_id_45600_45598.tsv", sep='\t', index=False)


34200 : 2017-03-26 20:40:30
35000 : 2017-03-26 20:55:41
36000 : 2017-03-26 21:14:10
flat_chemical      CID100160036
stereo_chemical    CID006918193
source_name                  PC
source_id              62707670
Name: 36457, dtype: object
37000 : 2017-03-26 21:32:47
flat_chemical      CID100160036
stereo_chemical    CID006918193
source_name                  PC
source_id              66545953
Name: 37692, dtype: object
38000 : 2017-03-26 21:51:12
39000 : 2017-03-26 22:09:36
<urlopen error [Errno 110] Connection timed out> https://www.ebi.ac.uk/unichem/rest/src_compound_id/67392947/22/2
  cnt = 39098, error time : 2017-03-26 22:13:31
    Retrying in 0 seconds...
40000 : 2017-03-26 22:30:03
41000 : 2017-03-26 22:48:38
42000 : 2017-03-26 23:07:03
flat_chemical      CID100160036
stereo_chemical    CID006918193
source_name                  PC
source_id              70682942
Name: 42774, dtype: object
flat_chemical      CID109818231
stereo_chemical    CID020111936
source_name                 

In [78]:
pd.DataFrame(data = [[stitch_id_flat, stitch_id_stereo, pubchem_id, db_id]], columns = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID'])

,STITCH_compound_id_flat,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID100005672,CID021780850,21780850,"[{""src_compound_id"":""DB00194""}]"


In [74]:
features = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID']
resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
resultTsv.writeheader()


3635